Data Source: https://www.uci.org/mountain-bike/results

In [1]:
import requests
import json
import os

In [2]:
URL_BASE = 'https://dataride.uci.ch/iframe/'

# All competitions
URL_COMPETITIONS = URL_BASE + 'Competitions/' 

# Races in a competition
URL_RACES = URL_BASE + 'Races/'

# Events in a race
URL_EVENTS = URL_BASE + 'Events/'

# Results for event
URL_RESULTS = URL_BASE + 'Results/'

In [3]:
DICIPLINE_ID_MOUNTAIN_BIKE = '7'

RACE_TYPE_ID_DOWNHILL = '19'
RACE_TYPE_ID_ENDURO = '122'
RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC = '92'

SEASON_ID_YEAR_MAP = {
    2020: '129',
    2019: '128',
    2018: '123',
    2017: '22',
    2016: '12',
    2015: '4',
    2014: '102',
    2013: '103',
    2012: '104',
    2011: '105',
    2010: '106',
    2009: '107',
}

COMPETITION_CLASS_CODE_WORLD_CHAMPS = 'CM'
COMPETITION_CLASS_CODE_WORLD_CUP = 'CDM'
COMPETITION_CLASS_CODE_ENDURO_WORLD_SERIES = '3'

CATEGORY_CODE_MEN_ELITE = 'Men Elite'
CATEGORY_CODE_WOMEN_ELITE = 'Women Elite'

RACE_TYPE_CODE_DHI = 'DHI'
RACE_TYPE_CODE_ENDURO = 'END'
RACE_TYPE_CODE_XCO = 'XCO'

RACE_TYPE_ID_TO_CODE_MAP = {
    RACE_TYPE_ID_DOWNHILL: RACE_TYPE_CODE_DHI,
    RACE_TYPE_ID_ENDURO: RACE_TYPE_CODE_ENDURO,
    RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC: RACE_TYPE_CODE_XCO
}

In [4]:
os.mkdir('../data')

In [5]:
def getCompetitions(race_type_id: str, year: int):
    request_body = {
        "disciplineId": DICIPLINE_ID_MOUNTAIN_BIKE,
        "take":"400",
        "skip":"0",
        "page":"1",
        "pageSize":"400",
        "sort": [{"field": "StartDate", "dir": "desc"}],
        "filter": {
            "filters": [
                {"field": "RaceTypeId", "value": race_type_id},
                {"field": "SeasonId", "value": SEASON_ID_YEAR_MAP[year]}
           ]
        }
    }
    response = requests.post(URL_COMPETITIONS, json=request_body).json()
    if len(response['data']) < response['total']:
        print('DID NOT GET ALL COMPETITIONS')
    return response['data']

In [6]:
def getRaces(competition_id: str):
    request_body = {
        "disciplineId": DICIPLINE_ID_MOUNTAIN_BIKE,
        "competitionId": competition_id,
        "take":"400",
        "skip":"0",
        "page":"1",
        "pageSize":"400"
    }
    response = requests.post(URL_RACES, json=request_body).json()
    if len(response['data']) < response['total']:
        print('DID NOT GET ALL RACES')
    return response['data']

In [7]:
def getEvents(race_id: str):
    request_body = {
        "disciplineId": DICIPLINE_ID_MOUNTAIN_BIKE,
        "raceId": race_id
    }
    return requests.post(URL_EVENTS, json=request_body).json()

In [8]:
def getResults(event_id: str):
    request_body = {
        "disciplineId": DICIPLINE_ID_MOUNTAIN_BIKE,
        "eventId": event_id,
        "take":"400",
        "skip":"0",
        "page":"1",
        "pageSize":"400"
    }
    response = requests.post(URL_RESULTS, json=request_body).json()
    if len(response['data']) < response['total']:
        print('DID NOT GET ALL RESULTS')
    return response['data']


For each year, retrieve the competitions for each of our chosen mountain bike disciplines (downhill, enduro and cross country olympic)

In [9]:
competitions = {
    RACE_TYPE_ID_DOWNHILL: {},
    RACE_TYPE_ID_ENDURO: {},
    RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC: {}
}

In [10]:
for year in SEASON_ID_YEAR_MAP:
    competitions[RACE_TYPE_ID_DOWNHILL][year] = getCompetitions(RACE_TYPE_ID_DOWNHILL, year)
    competitions[RACE_TYPE_ID_ENDURO][year] = getCompetitions(RACE_TYPE_ID_ENDURO, year)
    competitions[RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC][year] = getCompetitions(RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC, year)

In [11]:
competitions_json = json.dumps(competitions)

with open("../data/competitions.json","w") as f:
    f.write(competitions_json)

In [12]:
with open('../data/competitions.json') as f:
    competitions = json.load(f)

Filter the competitions:

For downhill and XCO we filter for only world cup and world championship competitions.

For enduro we filter for EWS competitions.

In [13]:
filtered_competitions = {
    RACE_TYPE_ID_DOWNHILL: {},
    RACE_TYPE_ID_ENDURO: {},
    RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC: {}
}

In [14]:
for race_type in filtered_competitions:
    for year in competitions[race_type]:
        filtered_competitions[race_type][year] = []
        for competition in competitions[race_type][year]:
            if race_type in [RACE_TYPE_ID_DOWNHILL, RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC]:
                if competition['ClassCode'] in [COMPETITION_CLASS_CODE_WORLD_CHAMPS, COMPETITION_CLASS_CODE_WORLD_CUP]:
                    filtered_competitions[race_type][year].append(competition)
            if race_type == RACE_TYPE_ID_ENDURO:
                if competition['ClassCode'] == COMPETITION_CLASS_CODE_ENDURO_WORLD_SERIES:
                    filtered_competitions[RACE_TYPE_ID_ENDURO][year].append(competition)

For each filtered competition, fetch the races

In [15]:
for race_type in filtered_competitions:
    for year in filtered_competitions[race_type]:
        for competition in filtered_competitions[race_type][year]:
            races = getRaces(competition['CompetitionId'])
            competition['races'] = races

In [16]:
competitions_with_races_json = json.dumps(filtered_competitions)

with open("../data/competitions_with_races.json","w") as f:
    f.write(competitions_with_races_json)

In [17]:
with open('../data/competitions_with_races.json') as f:
    competitions_with_races = json.load(f)

For each race (excluding qualifying), fetch the events

In [18]:
for race_type in competitions_with_races:
    for year in competitions_with_races[race_type]:
        for competition in competitions_with_races[race_type][year]:
            for race in competition['races']:
                race['events'] = {}
                for category_code in [CATEGORY_CODE_MEN_ELITE, CATEGORY_CODE_WOMEN_ELITE]:
                    if race['CategoryCode'] == category_code and race['RaceTypeCode'] == RACE_TYPE_ID_TO_CODE_MAP[race_type]:
                        if (race_type == RACE_TYPE_ID_DOWNHILL and 'qualifying' not in race['RaceName'].lower()) or race_type == RACE_TYPE_ID_CROSS_COUNTRY_OLYMPIC or race_type == RACE_TYPE_ID_ENDURO:
                            events = getEvents(race['Id'])
                            if (len(events) > 1):
                                print('MORE THAN ONE EVENT')
                            race['events'][category_code] = events[0]

In [19]:
competitions_with_races_and_events_json = json.dumps(competitions_with_races)

with open("../data/competitions_with_races_and_events.json","w") as f:
    f.write(competitions_with_races_and_events_json)

In [20]:
with open('../data/competitions_with_races_and_events.json') as f:
    competitions_with_races_and_events = json.load(f)

For each event, fetch the results

In [21]:
for race_type in competitions_with_races_and_events:
    for year in competitions_with_races_and_events[race_type]:
        for competition in competitions_with_races_and_events[race_type][year]:
            for race in competition['races']:
                for category_code in [CATEGORY_CODE_MEN_ELITE, CATEGORY_CODE_WOMEN_ELITE]:
                    if category_code in race['events']:
                        event_id = race['events'][category_code]['EventId']
                        results = getResults(event_id)
                        race['events'][category_code]['results'] = results

In [22]:
competitions_with_races_and_events_and_results_json = json.dumps(competitions_with_races_and_events)

with open("../data/competitions_with_races_and_events_and_results.json","w") as f:
    f.write(competitions_with_races_and_events_and_results_json)